### Header ### 

In [2]:
import numpy as np 
import pandas as pd
import sklearn as sk 

In [3]:
df = pd.read_csv(
    "train_data/cntrbtrs_clnts_ops_trn.csv",
    encoding='windows-1251',
    sep=';',
    low_memory=False  #  это чтобы не было предупреждения о том, что в 18 столбце содержатся данные разных типов
)

df[:1000].to_csv("train_data/small_dataframe", encoding='windows-1251', sep=';')

### Разделение на признаки и целевую переменную ###

In [27]:
y = df['erly_pnsn_flg']  # Целевая переменная
# Удаление нерелевантных и высококардинальных признаков
X = df.drop(columns=["erly_pnsn_flg"])
# X = new_df

X

,Unnamed: 0,prsnt_age,pnsn_age,rgn,pstl_code,okato,age_at_contract,age_vs_pension,mvmnt_type_ratio_0.0,mvmnt_type_ratio_1.0,avg_sum_3M,std_sum_3M,transaction_freq_3M,avg_sum_6M,std_sum_6M,transaction_freq_6M,gen
0,204992,60,55,КРАСНОДАРСКИЙ КРАЙ,353860,3.000000e+09,48,-7,0.812500,0.187500,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
1,155909,61,55,БАШКОРТОСТАН РЕСП,450006,8.000000e+10,43,-12,0.925926,0.074074,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
2,328282,62,55,БАШКОРТОСТАН РЕСП,452684,8.000000e+10,51,-4,0.831111,0.168889,1882.763333,1418.879572,9.0,1882.763333,1418.879572,9.0,0
3,160578,66,60,МОСКОВСКАЯ ОБЛ,142502,4.600000e+10,50,-10,0.920000,0.080000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1
4,531537,62,55,МОСКВА Г,111625,4.500000e+10,53,-2,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532820,151348,60,55,САРАТОВСКАЯ ОБЛ,410003,6.300000e+10,48,-7,0.833333,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
532821,76364,67,55,ЛЕНИНГРАДСКАЯ ОБЛ,187322,4.100000e+10,56,1,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
532822,308802,59,56,КУРГАНСКАЯ ОБЛ,640022,3.700000e+10,47,-9,0.750000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
532823,166137,62,55,ИРКУТСКАЯ ОБЛ,669516,2.500000e+10,46,-9,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0


Наблюдается дисбаланс классов

In [28]:
from collections import Counter
Counter(y)

Counter({0: 513448, 1: 19377})

In [29]:
categ_features = [i for i, c in enumerate(X.dtypes) if c == np.dtype('object')]

print(categ_features)

X

[3, 4]


,Unnamed: 0,prsnt_age,pnsn_age,rgn,pstl_code,okato,age_at_contract,age_vs_pension,mvmnt_type_ratio_0.0,mvmnt_type_ratio_1.0,avg_sum_3M,std_sum_3M,transaction_freq_3M,avg_sum_6M,std_sum_6M,transaction_freq_6M,gen
0,204992,60,55,КРАСНОДАРСКИЙ КРАЙ,353860,3.000000e+09,48,-7,0.812500,0.187500,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
1,155909,61,55,БАШКОРТОСТАН РЕСП,450006,8.000000e+10,43,-12,0.925926,0.074074,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
2,328282,62,55,БАШКОРТОСТАН РЕСП,452684,8.000000e+10,51,-4,0.831111,0.168889,1882.763333,1418.879572,9.0,1882.763333,1418.879572,9.0,0
3,160578,66,60,МОСКОВСКАЯ ОБЛ,142502,4.600000e+10,50,-10,0.920000,0.080000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1
4,531537,62,55,МОСКВА Г,111625,4.500000e+10,53,-2,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532820,151348,60,55,САРАТОВСКАЯ ОБЛ,410003,6.300000e+10,48,-7,0.833333,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
532821,76364,67,55,ЛЕНИНГРАДСКАЯ ОБЛ,187322,4.100000e+10,56,1,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
532822,308802,59,56,КУРГАНСКАЯ ОБЛ,640022,3.700000e+10,47,-9,0.750000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
532823,166137,62,55,ИРКУТСКАЯ ОБЛ,669516,2.500000e+10,46,-9,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0


Семплирование датасета

In [30]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

alpha = 0.3
X_pretrain, X_test, y_pretrain, y_test = train_test_split(X, y, train_size=.7)

# smote = SMOTE(random_state=42)
# X_train, y_train = smote.fit_resample(X_pretrain, X_pretest)

X_train, y_train = X_pretrain, y_pretrain

X_train.shape, y_train.shape

((372977, 17), (372977,))

### Создание модели ### 

In [31]:
from Model import Model

model = Model(
    iterations=200,
    learning_rate=0.4,
    depth=10,
    task_type="GPU",
    devices='0'
)

### Обучение ###

In [32]:
model.fit(X_train, y_train, categ_features, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0367895	total: 13.9ms	remaining: 2.77s
1:	learn: 0.0059156	total: 71.8ms	remaining: 7.11s
2:	learn: 0.0026344	total: 128ms	remaining: 8.43s
3:	learn: 0.0020767	total: 200ms	remaining: 9.78s
4:	learn: 0.0018587	total: 257ms	remaining: 10s
5:	learn: 0.0017699	total: 314ms	remaining: 10.2s
6:	learn: 0.0016895	total: 371ms	remaining: 10.2s
7:	learn: 0.0016175	total: 428ms	remaining: 10.3s
8:	learn: 0.0015852	total: 491ms	remaining: 10.4s
9:	learn: 0.0015198	total: 549ms	remaining: 10.4s
10:	learn: 0.0014793	total: 609ms	remaining: 10.5s
11:	learn: 0.0014327	total: 672ms	remaining: 10.5s
12:	learn: 0.0013974	total: 738ms	remaining: 10.6s
13:	learn: 0.0013769	total: 805ms	remaining: 10.7s
14:	learn: 0.0013553	total: 868ms	remaining: 10.7s
15:	learn: 0.0013178	total: 931ms	remaining: 10.7s
16:	learn: 0.0012718	total: 993ms	remaining: 10.7s
17:	learn: 0.0012336	total: 1.06s	remaining: 10.7s
18:	learn: 0.0012186	total: 1.12s	remaining: 10.7s
19:	learn: 0.0011944	total: 1.18s	remaini

### Метрика ###

In [33]:
from sklearn.metrics import f1_score, accuracy_score

y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average="binary")
acc = accuracy_score(y_test, y_pred)

f1, acc 

(0.9958577839143942, 0.9996997147289926)

### Save model ###

In [34]:
model.save("best_tree")